<a href="https://colab.research.google.com/github/OJB-Quantum/Self-Avoiding-Fractals/blob/main/Fibonacci%20Fractal%20Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install svgwrite
!pip install math
!pip install ezdxf

ERROR: Could not find a version that satisfies the requirement math (from versions: none)
ERROR: No matching distribution found for math
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 51.3 MB/s eta 0:00:00


In [5]:
import svgwrite
import math
import ezdxf

In [ ]:
# Connects line segments to form self-avoiding fractal curves based on the paper "The Fibonacci Word Fractal" (https://hal.science/file/index/docid/367972/filename/The_Fibonacci_word_fractal.pdf)
# The curves are generated as SVG drawings.

def generate_fibonacci_word(n):
    """Generate the Fibonacci word sequence up to the nth term."""
    if n == 1:
        return "1"
    elif n == 2:
        return "0"
    else:
        f1 = "1"
        f2 = "0"
        for _ in range(3, n + 1):
            fn = f2 + f1
            f1, f2 = f2, fn
        return fn

def draw_fibonacci_fractal(dwg, start_pos, length, angle, word, line_thickness):
    """Draw the Fibonacci word fractal based on the given word, while tracking boundaries."""
    current_pos = start_pos
    current_angle = angle

    # Boundary tracking
    min_x, min_y = start_pos
    max_x, max_y = start_pos

    for i, char in enumerate(word):
        # Calculate next position
        next_pos = (
            current_pos[0] + length * math.cos(math.radians(current_angle)),
            current_pos[1] + length * math.sin(math.radians(current_angle)),
        )

        # Update boundaries
        min_x = min(min_x, next_pos[0])
        min_y = min(min_y, next_pos[1])
        max_x = max(max_x, next_pos[0])
        max_y = max(max_y, next_pos[1])

        # Draw the line segment
        dwg.add(dwg.line(start=current_pos, end=next_pos, stroke=svgwrite.rgb(0, 0, 0, '%'), stroke_width=line_thickness))

        # Update current position
        current_pos = next_pos

        # Turn based on the Fibonacci word rule
        if char == '0':
            if (i + 1) % 2 == 0:  # even index in 1-based counting
                current_angle -= 90
            else:
                current_angle += 90
        # No turn for '1'

    return min_x, min_y, max_x, max_y

def create_fibonacci_word_fractal(num_segments, line_thickness, filename="fibonacci_fractal.svg"):
    # Generate the Fibonacci word corresponding to the number of segments
    fibonacci_word = generate_fibonacci_word(num_segments)

    # Initial position and angle
    start_pos = (0, 0)
    initial_angle = 0
    segment_length = 10  # You can adjust the length of each segment here

    # Initialize SVG drawing
    dwg = svgwrite.Drawing(filename, profile='tiny')

    # Draw the fractal and get boundaries
    min_x, min_y, max_x, max_y = draw_fibonacci_fractal(dwg, start_pos, segment_length, initial_angle, fibonacci_word, line_thickness)

    # Adjust SVG canvas size based on boundaries
    canvas_width = max_x - min_x + 2 * line_thickness
    canvas_height = max_y - min_y + 2 * line_thickness
    dwg.viewbox(min_x - line_thickness, min_y - line_thickness, canvas_width, canvas_height)

    # Save the drawing
    dwg.save()

# Example usage:
# User specifies number of segments and line thickness
# There are self-similarities between F_n and F_n-3 curves according to the Monnerot-Dumaine paper
# This suggests that every 3 iterations there is a self-similarity
# Keep in mind that 26 iterations is the limit for the fractal density when run in Google Colab

num_segments = 23  # Replace this with user input
line_thickness = 5  # Replace this with user input in pixels

# Create the fractal and save as SVG
create_fibonacci_word_fractal(num_segments, line_thickness)

In [7]:
# The following script generates a continous curve of the Fibonacci word fractal in DXF format.

def generate_fibonacci_word(n):
    """Generate the Fibonacci word sequence up to the nth term."""
    if n == 1:
        return "1"
    elif n == 2:
        return "0"
    else:
        f1 = "1"
        f2 = "0"
        for _ in range(3, n + 1):
            fn = f2 + f1
            f1, f2 = f2, fn
        return fn

def draw_fibonacci_fractal(doc, msp, start_pos, length, angle, word, line_thickness):
    """Draw the Fibonacci word fractal based on the given word, while tracking boundaries."""
    current_pos = start_pos
    current_angle = angle

    # Boundary tracking
    min_x, min_y = start_pos
    max_x, max_y = start_pos

    for i, char in enumerate(word):
        # Calculate next position
        next_pos = (
            current_pos[0] + length * math.cos(math.radians(current_angle)),
            current_pos[1] + length * math.sin(math.radians(current_angle)),
        )

        # Update boundaries
        min_x = min(min_x, next_pos[0])
        min_y = min(min_y, next_pos[1])
        max_x = max(max_x, next_pos[0])
        max_y = max(max_y, next_pos[1])

        # Draw the line segment
        msp.add_line(current_pos, next_pos, dxfattribs={'lineweight': line_thickness})

        # Update current position
        current_pos = next_pos

        # Turn based on the Fibonacci word rule
        if char == '0':
            if (i + 1) % 2 == 0:  # even index in 1-based counting
                current_angle -= 90
            else:
                current_angle += 90
        # No turn for '1'

    return min_x, min_y, max_x, max_y

def create_fibonacci_word_fractal_dxf(num_segments, line_thickness, filename="fibonacci_fractal.dxf"):
    # Generate the Fibonacci word corresponding to the number of segments
    fibonacci_word = generate_fibonacci_word(num_segments)

    # Initialize DXF drawing
    doc = ezdxf.new(dxfversion='R2010')
    msp = doc.modelspace()

    # Initial position and angle
    start_pos = (0, 0)
    initial_angle = 0
    segment_length = 10  # You can adjust the length of each segment here

    # Draw the fractal and get boundaries
    min_x, min_y, max_x, max_y = draw_fibonacci_fractal(doc, msp, start_pos, segment_length, initial_angle, fibonacci_word, line_thickness)

    # Adjust DXF extents (optional, just for reference, does not affect the actual drawing)
    doc.header['$EXTMIN'] = (min_x, min_y, 0)
    doc.header['$EXTMAX'] = (max_x, max_y, 0)

    # Save the drawing
    doc.saveas(filename)

# Example usage:
# User specifies number of segments and line thickness
num_segments = 14  # Replace this with user input
line_thickness = 4  # Replace this with user input in mm (for DXF)

# Create the fractal and save as DXF
create_fibonacci_word_fractal_dxf(num_segments, line_thickness)